In [ ]:
#Import packages
import pandas as pd
import requests
import json

from config import gkey

In [ ]:
#load crash data
file_path = 'Resources/CRASH DATA SET'

crash_csv = pd.read_csv(file_path)

place_holder_df = pd.DataFrame()

In [ ]:
#set list of lat lng to feed into google

lat = df['Start_lat']
lng = df['Start_lng']

for row in crash_csv:
    #lat = df['Start_lat']
    #lng = df['Start_lng']
    google_path = f'{lat},{lng}'
    params = {'Key': gkey,
         'Path' : google_path}
    base_url = 'https://roads.googleapis.com/v1/speedLimits?'

    response = requests.get(base_url, params=params)
    
    roads_data = response.json()
    
    
    

In [ ]:
# convert to json
roads_data = response.json()



In [ ]:
# Jon's code starts here.

In [ ]:
import gmaps

In [ ]:
# Store DataFrame/.csv path. Change to df if .csv.
# Above: load crash data #
**df**

# Add "Speed Limit" column to the DataFrame.
**df**["Speed Limit"] = ""

# Set parameters for speed limit search.
params = {
    "units": "MPH",
    "key": gkey
}

# Set a variable to track each iteration.
Request_number = 0

# Use for loop to locate the speed limits.
for index, row in **df**.iterrows():
    
    # Get Lat & Lng from DataFrame.
    lat = row["Start_Lat"]
    lng = row["Start_Lng"]
    
    # Change location each iteration while leaving original params in place.
    params["path"] = f"{lat},{lng}"
    
    # Set base URL.
    base_url = "https://roads.googleapis.com/v1/speedLimits?"
    
    # Make requests and print URL.
    speed = requests.get(base_url, params=params)
    
    # Convert to json
    speed = speed.json()
    
    # In case there may be missing speed limits, use try-except to skip any missing data.
    try:
        **df**.loc[index, "Speed Limit"] = speed["speedLimits:"][0]["speedLimit:"]
        Request_number = Request_number + 1
        time.sleep(.1)
        print(f"Processed request number {Request_number}")
    except:
        Request_number = Request_number + 1
        print("Speed limit not found. Skipping...")
        pass
        

In [ ]:
#Jon's code ends here.